# インポート, csvファイルダウンロード
 (まずkaggle.jsonファイルをカレントディレクトリに置いて置く)

In [8]:
#kaggleAPIトークンファイルを設定する
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
from kaggle import KaggleApi
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from torch import nn, optim
from torchsummary import torchsummary
import zipfile
import os

#kaggleAPIをインスタンス化して認証する
api = KaggleApi()
api.authenticate()

In [10]:
#kaggleAPI経由でファイルをダウンロードして解凍する
compe_name = 'h-and-m-personalized-fashion-recommendations'
csv_fname = [f for f in api.competition_list_files(compe_name) if '.csv' in f.name]
for f in csv_fname:
  api.competition_download_file(compe_name,f.name)
  with zipfile.ZipFile(f.name+'.zip') as zf:
    zf.extractall()
  os.remove(f.name+'.zip')
for i, f in enumerate(csv_fname): print(i, f)

100%|██████████| 97.9M/97.9M [00:00<00:00, 125MB/s]


100%|██████████| 50.3M/50.3M [00:00<00:00, 125MB/s]


100%|██████████| 584M/584M [00:04<00:00, 148MB/s]


100%|██████████| 4.26M/4.26M [00:00<00:00, 60.1MB/s]

0 customers.csv
1 sample_submission.csv
2 transactions_train.csv
3 articles.csv


# Dataframeの作成

In [11]:
transactions = pd.read_csv('transactions_train.csv')
customers = pd.read_csv('customers.csv').set_index('customer_id')
articles = pd.read_csv('articles.csv').set_index('article_id')

In [12]:
#結合処理が重すぎるのでとりあえず取引データを10,000件サンプリング
transactions_lt = transactions.sample(10000)
#customerIDごとに取引データをグルーピングして、服カテゴリデータを結合する
ap_info = ['product_code','product_type_no','colour_group_code','garment_group_no']
transactions_by_customer = transactions_lt.groupby('customer_id').apply(lambda d:[articles.loc[id][ap_info].to_list() for id in d['article_id']])
display(transactions_by_customer)